<a href="https://colab.research.google.com/github/shubham8899/arrowhead-take-home-assignment/blob/main/Summarization_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prerequisites
Run all cells in this section in order

In [ ]:
!pip install transformers==4.28.0
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.6 MB/s eta 0:00:

In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import AutoModelForSeq2SeqLM, AutoConfig, AutoModel, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from sklearn.model_selection import train_test_split

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MAX_LEN_ARTICLE = 1024
MAX_LEN_SUMMARY = 512

In [ ]:
model_name = 'facebook/bart-large-cnn'
# config = AutoConfig.from_pretrained(model_name)
# model =  AutoModel.from_config(config)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(DEVICE)

In [ ]:
#CHANGE WEIGHTS FILE TO USE APPROPRIATE MODEL
model.load_state_dict(torch.load("/content/drive/MyDrive/Arrowhead Assignment/BART_4_ep.pth"))

<All keys matched successfully>

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Arrowhead Assignment/dataset.csv', index_col=0)

In [ ]:
# Converting pandas df to huggingface dataset
dataset = Dataset.from_pandas(dataset)
# 25% for testing
dataset = dataset.train_test_split(test_size=0.25, shuffle=False)

In [ ]:
generations_args = {'length_penalty': 0.8, 'num_beams': 8, 'max_length': MAX_LEN_SUMMARY}

In [ ]:
def generate_summary(article, generations_args, model, device = DEVICE):
    input = tokenizer(article, max_length=MAX_LEN_ARTICLE,  truncation=True, padding="max_length", return_tensors="pt")
    summary = model.generate(input_ids=input["input_ids"].to(device), attention_mask=input["attention_mask"].to(device), 
                         length_penalty=0.8, num_beams=8, max_length=MAX_LEN_SUMMARY)
    # summary = tokenizer.decode(summary)      
    decoded_summary = tokenizer.decode(summary[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print('Summary:', decoded_summary)

## Generate Summary
- Run all cells above to load all prerequisites for inference.
- Use this function with any text to generate summary.

In [ ]:
article = input("Input Article: ")
generate_summary(article, generations_args, model)

Input Article: The BCP-47 code for English as generally spoken in the United States is en-US and the BCP-47 code for English as generally spoken in the United Kingdom is en-GB. It is unknown if other varieties of English are represented in the data.
Summary: It is unknown if other varieties of English are represented in the data.The BCP-47 code for English as generally spoken in the United States is en-US and the BCP to the B CP-47 codes for language in the UK are en-GB.
